# Código

In [1]:
import pandas as pd
import sklearn
import scipy.io as sio
import numpy as np
from os import getcwd
from os.path import join 
import matplotlib.pyplot as plt
from scipy.fftpack import rfft, rfftfreq
from lvm_read import read
from sklearn.datasets import make_regression, make_classification, make_blobs
from sklearn import linear_model, decomposition, preprocessing,metrics, model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt
from sklearn import metrics
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns
from tqdm.notebook import tqdm_notebook as tqdm
import seaborn as sns
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

c:\Program Files\Python38\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [41]:
df=[]
df_all=[]
df_act=[]
cn=[]
cn_all=[]
#Params Calculo da FFT
Ts = 6 * 1e-4 # sampling time
sample_rate=1666 #Hertz
time = (np.linspace(1,sample_rate,sample_rate) - 1) * Ts
sample_size = len(time)
x = rfftfreq(sample_size, 1 / sample_rate)
#Params Download Data.

case_scheme = ["R","H","L"]
CH_numbers = 8


for jj, case in enumerate(case_scheme):
    for i in tqdm(range (1,21),desc=f"Case {case}"):
        if i>20:
            ij=i-20
            sensor_scheme="2"
        else:
            ij=i
            sensor_scheme ="1" # Sensor Scheme Choosed

        string="white_noise_"+str(ij)+".lvm"
        try:
            d=read(join(getcwd(),f'Case_{case}_(+25)/Case_{case}_(+25)_{sensor_scheme}/',string))
            
            d[0]['Channel names'].pop()
            df=pd.DataFrame(d[0]['data'],columns=d[0]['Channel names'])
            df_freq = []
            for ii in range(1,CH_numbers+1):
                channel_name = f"Ch{ii}"
                sensor_data = df[channel_name]
                normalized_sensor = np.int16(((sensor_data-sensor_data.mean())/ sensor_data.max()) * 32767) # Constant 32767 due to 16int max bit size
                df_freq_act=(np.abs(rfft(normalized_sensor,n=sample_rate)[1:])**2)/(x[1:]*32767)

                if ii==1 or df_freq==[]:
                    df_freq=df_freq_act
                else:
                    df_freq=np.concatenate([df_freq, df_freq_act])
            if i==1:
                df_act=np.reshape(df_freq,(1,-1))
                cn = pd.DataFrame([f"{sensor_scheme}_{case}_{str(ij)}_target_{str(jj+1)}"],columns=['case'])
            else:
                df_act=np.row_stack([df_act,np.reshape(df_freq,(1,-1))])
                cn = pd.concat([cn,pd.DataFrame([f"{sensor_scheme}_{case}_{str(ij)}_target_{str(jj+1)}"],columns=['case'])])
        except:
            continue
    df_act=pd.DataFrame(df_act)
    N,Chno=df_act.shape
    y_act=pd.DataFrame(N*[f'{jj+1}'],columns=['target'])

    if jj == 0:
        df_all = df_act
        y_all = y_act
        cn_all = cn
    else:
        df_all=pd.concat([df_all,df_act],axis=0,ignore_index=True)
        y_all=pd.concat([y_all,y_act],axis=0,ignore_index=True)
        cn_all=pd.concat([cn_all,cn],axis=0,ignore_index=True)


Case R:   0%|          | 0/20 [00:00<?, ?it/s]

Case H:   0%|          | 0/20 [00:00<?, ?it/s]

Case L:   0%|          | 0/20 [00:00<?, ?it/s]

In [42]:
#plt.semilogy(x[1:],abs(df_freq_act))

y_all.value_counts()


target
2         24
1         20
3          5
dtype: int64

In [ ]:
# PCA

pca = PCA().fit(df_all.to_numpy())
plt.rcParams["figure.figsize"] = (12,6)
fig, ax = plt.subplots()
yi = np.cumsum(pca.explained_variance_ratio_)
xi = np.arange(1, np.size(yi)+1, step=1)

plt.ylim(0.0,1.1)
plt.plot(xi, yi, linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, np.size(yi)+1, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=14)
plt.axhline(y=0.99, color='g', linestyle='-')
plt.text(0.5, 0.95, '99% cut-off threshold', color = 'green', fontsize=14)
plt.axhline(y=0.80, color='black', linestyle='-')
plt.text(0.5, 0.75, '80% cut-off threshold', color = 'black', fontsize=14)

ax.grid(axis='x')
plt.show()

In [ ]:
df_all_PCA = pd.DataFrame(PCA(n_components=7).fit_transform(df_all.to_numpy()))#,columns=[f'{x} Component' for x in range(1,np.size(yi)+1)])
df_parallel = pd.concat([df_all_PCA,y_all],axis=1)
import plotly.express as px
fig = pd.plotting.parallel_coordinates(df_parallel,"target",color=["lime", "tomato","dodgerblue"]) # px.parallel_coordinates(df_parallel,)


In [ ]:
df_melted = df_parallel.melt(id_vars=['target'])
sns.boxplot(data = df_melted[df_melted['variable']<=10],x='variable',y='value',hue='target')

In [ ]:
sns.pairplot(df_parallel,hue='target',height=1)

In [ ]:
df_TSNE = pd.DataFrame(TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(df_parallel[df_parallel.columns[0:-1]]))
df_TSNE = pd.concat([df_TSNE,df_parallel['target']],axis=1)
fig4=sns.pairplot(df_TSNE[(df_TSNE['target']!='3')],hue='target',height=4)
# fig4._legend.remove()
# handles = fig4._legend_data.values()
# labels = case_scheme
# fig4.fig.legend(handles=handles, labels=labels, loc='lower center', ncol=3,title='Categorias')
# fig4.fig.subplots_adjust(top=0.95, bottom=0.15)
# fig4.fig.suptitle('Dataset transformado por T-SNE e classificado pelo Label conhecido',fontsize=15)

In [ ]:
df_TSNE['target'].value_counts()